In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import os
import json

In [2]:
def lon_lat_append(folder, createFolder, year, month=None, encoding = 'cp949', ):
    
    # 폴더 생성 경로
    fol = './전처리/wi-fi/path/'
        
    # 동일한 위경도값을 보유한 포인트를 1개의 개소로 측정
    # 위경도 csv 파일 읽기
    wk = pd.read_csv('./전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])
    
    # MAC 코드화 딕셔너리 불러오기
    with open('mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')

    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))
    
    
    # 사용되지 않은 Wi-Fi, 목록에서 제거
    for n in wk.index:
        if n not in mac_code.values():
            wk.drop(n, axis=0, inplace=True)

    # 위도로 정렬 (같은 위도 묶기)
    wk.sort_values('위도', inplace=True)
    
    # 이전 Mac 주소를 담을 변수
    f_mac = None

    # 동일한 위경도를 보유한 Mac 주소 리스트
    repeat = []

    # repeat 리스트를 모은 리스트
    repeat_all = []

    for n in wk.index:
        # 첫행 Skip
        if n == wk.iloc[0].name:
            f_mac = n
            continue
        # 이전 행 위도와 같다면 append
        if wk.loc[f_mac]['위도'] == wk.loc[n]['위도']:
            repeat.append(f_mac)
            repeat.append(n)

        # 전체 리스트 
        else:
            # 빈 리스트는 무시
            if len(repeat) == 0:
                f_mac = n
                continue
            repeat_all.append(list(set(repeat)))
            repeat = []
        f_mac = n
    
    with open('mac_code.json', 'r') as f :
        mac_code = json.load(f, encoding='utf-8')

    repeat_num_all = []
    repeat_num = []
    for n in repeat_all:
        for i in range(len(n)):
            repeat_num.append(mac_code[n[i]])
        repeat_num_all.append(repeat_num)
        repeat_num = []

    # key값과 values값 위치 변경
    mac_code = dict(zip(mac_code.values(),mac_code.keys()))

    for n in repeat_num_all:
        for i in range(len(n)):
            if i == 0:
                continue
            mac_code[n[i]] = mac_code[n[0]]
     
    # csv 파일명 list 생성
    csv_list = os.listdir(f'{folder}')
    
    # 병합을 위한 변수
    cnct_df = None
    
    # csv 파일명으로 변수 생성 
    for n in tqdm(csv_list):
        name = n.split('.')[0]
        df = pd.read_csv(f'{folder}{n}', encoding=encoding)
        
        # 위경도 컬럼 추가 
        df['main_X'] = None
        df['main_Y'] = None
        df['next_X'] = None
        df['next_Y'] = None
        
        for a in range(len(df)):
            
#             # MAC 주소 코드화 해제
#             df['MAC_add'][a] = mac_code[df['MAC_add'][a]]
#             df['Next_MAC'][a] = mac_code[df['Next_MAC'][a]]
            
            # Main MAC 주소 위경도 추가 (기준 MAC)
            df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
            df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
        
            # 다음 이동경로 MAC 주소 위경도 추가
            b = df['Next_MAC'][a]
            df['next_X'][a] = wk.loc[b]['경도']
            df['next_Y'][a] = wk.loc[b]['위도']
                            
        # 폴더 생성
        try:
            directory = fol + '위경도/' + createFolder
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
        
        if month != None:
            # 폴더 생성
            try:
                directory = fol + '위경도/' + createFolder + f'/{month}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
            df.to_csv(f'{directory}/{n}', encoding=encoding, index=False)

        else:
            # 생성한 폴더에 저장
            df.to_csv(f'{directory}/{n}', encoding=encoding, index=False)
                
        # 데이터프레임 병합
        if n == csv_list:
            cnct_df = df
            continue
        cnct_df = pd.concat([cnct_df, df])
    
    # 병합 데이터프레임 저장
    if month != None:
        
        # 폴더 생성
        try:
            directory = fol + '위경도/' + createFolder + f'/{month}/'
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
        cnct_df.to_csv(f'{directory}/path_{year}_{month}_all.csv', encoding=encoding, index=False)
        
    else:
        cnct_df.to_csv(f'{directory}/path_{year}_all.csv', encoding=encoding, index=False)

In [3]:
lon_lat_append('./전처리/wi-fi/path/count/2021/', 'count_2021_위경도', 2021)

  0%|          | 0/53 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [4]:
lon_lat_append('./전처리/wi-fi/path/count/2022/', 'count_2022_위경도', 2022)

  0%|          | 0/53 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [5]:
# 하위 디렉토리명 리스트화
def dir_list(year):
    
    year_path = f'./전처리/wi-fi/path/count/{year}년_월별/'
    
    # 글로벌 함수 사용
    # 지역변수에서 전역변수로 전환
    global month_list
    month_list = []
    
    # year_path 디렉토리 내에서 디렉토리만 추출
    for file in os.listdir(year_path):
        X = os.path.join(year_path, file)
        if os.path.isdir(X):
            month_list.append(file)

In [6]:
# 년도별 월간 csv 파일 처리
def month_lon_lat_append(year):
    dir_list(year)
    for n in tqdm(month_list):
        print(n)
        try:
            lon_lat_append(f'./전처리/wi-fi/path/count/{year}년_월별/{n}/', f'count_{year}_월별_위경도', year, month=n)
        except:
            continue

In [7]:
month_lon_lat_append(2021)

  0%|          | 0/12 [00:00<?, ?it/s]

01월


  0%|          | 0/53 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [8]:
month_lon_lat_append(2022)

  0%|          | 0/10 [00:00<?, ?it/s]

01월


0it [00:00, ?it/s]

02월


0it [00:00, ?it/s]

...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_X'][a] = wk.loc[df['MAC_add'][a]]['경도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['main_Y'][a] = wk.loc[df['MAC_add'][a]]['위도']
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14996\3484654340.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin